In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

from xgboost import XGBClassifier
import os

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
NROWS = None
# NROWS = 50000

In [3]:
train_identity = pd.read_csv('../input/train_identity.csv', nrows=NROWS)
train_transaction = pd.read_csv('../input/train_transaction.csv', nrows=NROWS)
train = train_transaction.merge(train_identity, how='left', on='TransactionID')

test_identity = pd.read_csv('../input/test_identity.csv', nrows=NROWS)
test_transaction = pd.read_csv('../input/test_transaction.csv', nrows=NROWS)
test = test_transaction.merge(test_identity, how='left', on='TransactionID')

sub = pd.read_csv('../input/sample_submission.csv', nrows=NROWS)

gc.enable()
del train_identity, train_transaction
del test_identity, test_transaction
gc.collect()

print("train.shape:", train.shape)
print("test.shape:", test.shape)
train.head(3)

train.shape: (590540, 434)
test.shape: (506691, 433)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [4]:
target = "isFraud"

In [5]:
test[target] = -1

In [6]:
df = train.append(test)
df.reset_index()

,index,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D7,D8,D9,DeviceInfo,DeviceType,M1,M2,M3,M4,M5,M6,M7,M8,M9,P_emaildomain,ProductCD,R_emaildomain,TransactionAmt,TransactionDT,TransactionID,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V11,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V12,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V13,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V14,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V27,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V28,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V29,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V3,V30,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V31,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V32,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V33,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,addr1,addr2,card1,card2,card3,card4,card5,card6,dist1,dist2,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,isFraud
0,0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,14.0,13.0,13.0,NaN,NaN,NaN,0.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN,NaN,W,NaN,68.500,86400,2987000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,117.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,117.000000,0.000000,0.0,0.000000,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,117.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,315.0,87.0,13926,NaN,150.0,discover,142.0,credit,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1.0,

In [7]:
df['uid'] = df["card1"].apply(lambda x: str(x)) + "_" + df["card2"].apply(lambda x: str(x)) +\
                "_" + df["card3"].apply(lambda x: str(x)) + "_" + df["card4"].apply(lambda x: str(x)) +\
                "_" + df["card5"].apply(lambda x: str(x)) + "_" + df["card6"].apply(lambda x: str(x)) +\
                "_" + df["addr1"].apply(lambda x: str(x)) + "_" + df["addr2"].apply(lambda x: str(x))


In [8]:
df["day"] = df["TransactionDT"] // (24 * 60 * 60)

In [38]:
feature_list = ["uid", target, "D15", "day", "TransactionDT", "TransactionID"]

In [39]:
df.loc[(df["D15"] == 1) & (df["day"] == 8) & (df["isFraud"] == 0), feature_list]

,uid,isFraud,D15,day,TransactionDT,TransactionID
27754,11839_490.0_150.0_visa_226.0_debit_123.0_87.0,0,1.0,8,693861,3014754
27959,15219_555.0_131.0_mastercard_166.0_debit_161.0...,0,1.0,8,697132,3014959
27974,2114_396.0_150.0_mastercard_224.0_debit_315.0_...,0,1.0,8,697426,3014974
27994,10776_512.0_150.0_visa_166.0_debit_194.0_87.0,0,1.0,8,697886,3014994
28009,10776_512.0_150.0_visa_166.0_debit_194.0_87.0,0,1.0,8,698172,3015009
28076,5700_122.0_150.0_mastercard_166.0_debit_204.0_...,0,1.0,8,699607,3015076
29015,3681_513.0_150.0_visa_166.0_debit_299.0_87.0,0,1.0,8,744995,3016015
29125,15651_417.0_150.0_visa_226.0_debit_330.0_87.0,0,1.0,8,746762,3016125
29241,10960_567.0_150.0_mastercard_117.0_debit_272.0...,0,1.0,8,748402,3016241
29351,9633_130.0_185.0_visa_138.0_debit_465.0_60.0,0,1.0,8,749814,3016351


In [11]:
df.loc[(df["D15"] == 1) & (df["day"] == 8) & (df["isFraud"] == 0), "uid"].values

array(['11839_490.0_150.0_visa_226.0_debit_123.0_87.0',
       '15219_555.0_131.0_mastercard_166.0_debit_161.0_60.0',
       '2114_396.0_150.0_mastercard_224.0_debit_315.0_87.0',
       '10776_512.0_150.0_visa_166.0_debit_194.0_87.0',
       '10776_512.0_150.0_visa_166.0_debit_194.0_87.0',
       '5700_122.0_150.0_mastercard_166.0_debit_204.0_87.0',
       '3681_513.0_150.0_visa_166.0_debit_299.0_87.0',
       '15651_417.0_150.0_visa_226.0_debit_330.0_87.0',
       '10960_567.0_150.0_mastercard_117.0_debit_272.0_87.0',
       '9633_130.0_185.0_visa_138.0_debit_465.0_60.0',
       '12098_555.0_131.0_visa_226.0_credit_430.0_13.0',
       '7919_194.0_150.0_mastercard_202.0_debit_441.0_87.0',
       '12695_490.0_150.0_visa_226.0_debit_325.0_87.0',
       '13534_105.0_150.0_visa_226.0_debit_110.0_87.0',
       '7919_194.0_150.0_mastercard_202.0_debit_441.0_87.0',
       '12501_490.0_150.0_visa_226.0_debit_204.0_87.0',
       '7815_161.0_150.0_mastercard_117.0_debit_203.0_87.0',
       '1588

In [12]:
df.loc[(df["uid"] == "11839_490.0_150.0_visa_226.0_debit_123.0_87.0") & (df["day"] == 7), feature_list]

# day有可能是 7-2,也有可能是7-2-1
# D15 = int(delta秒/3600/24) 不对 3,16 bad case
# D15 = round(delta秒/3600/24)


,uid,isFraud,D10,day,TransactionDT,TransactionID
24016,11839_490.0_150.0_visa_226.0_debit_123.0_87.0,0,0.0,7,607887,3011016
25033,11839_490.0_150.0_visa_226.0_debit_123.0_87.0,0,250.0,7,654709,3012033
26356,11839_490.0_150.0_visa_226.0_debit_123.0_87.0,0,279.0,7,674190,3013356


In [30]:
DAY = 1  # (1, 182+1)
D15 = 0  # (0, DAY)

In [42]:
df.loc[(df["D15"] == D15) & (df["day"] == DAY), feature_list]

,uid,isFraud,D15,day,TransactionDT,TransactionID
0,13926_nan_150.0_discover_142.0_credit_315.0_87.0,0,0.0,1,86400,2987000
1,2755_404.0_150.0_mastercard_102.0_credit_325.0...,0,0.0,1,86401,2987001
5,5937_555.0_150.0_visa_226.0_debit_272.0_87.0,0,0.0,1,86510,2987005
6,12308_360.0_150.0_visa_166.0_debit_126.0_87.0,0,0.0,1,86522,2987006
7,12695_490.0_150.0_visa_226.0_debit_325.0_87.0,0,0.0,1,86529,2987007
10,16496_352.0_117.0_mastercard_134.0_credit_nan_nan,0,0.0,1,86549,2987010
11,4461_375.0_185.0_mastercard_224.0_debit_nan_nan,0,0.0,1,86555,2987011
14,11839_490.0_150.0_visa_226.0_debit_226.0_87.0,0,0.0,1,86596,2987014
15,7055_555.0_150.0_visa_226.0_debit_315.0_87.0,0,0.0,1,86618,2987015
18,4663_490.0_150.0_visa_166.0_debit_184.0_87.0,0,0.0,1,86725,2987018


In [43]:
uid_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY), "uid"].values)
TransactionID_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY), "TransactionID"].values)


In [48]:
df.loc[(df["uid"] == uid_list[3]) & (df["day"] == DAY-D15), feature_list]

,uid,isFraud,D15,day,TransactionDT,TransactionID
6,12308_360.0_150.0_visa_166.0_debit_126.0_87.0,0,0.0,1,86522,2987006


In [26]:
df.loc[(df["uid"] == uid_list[2]) & (df["day"] == DAY-D15), feature_list]["isFraud"].mean()

0.0

In [27]:
df.loc[(df["uid"] == uid_list[2]) & (df["day"] == DAY-D15), feature_list]["isFraud"].sum()

0

In [ ]:
df.loc[(df["D15"] == D15) & (df["day"] == DAY), feature_list]

In [52]:
fraud_TransactionIDs = []
uid_D15 = []

### 训练集特征构造

In [57]:
# 如果是D15==0,一天内只有一笔交易的话,不能用

for DAY in tqdm_notebook(range(2, 182+1)): # 2, 182+1
    for D15 in range(1, DAY):
        uid_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY), "uid"].values)
        TransactionID_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY), "TransactionID"].values)
        
        for i in range(len(uid_list)):
            TransactionID_ = TransactionID_list[i]
            
            if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list].shape[0] != 0:
                
#                 print(df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list])
                mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].mean()
                sum_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].sum()
                
            else:
                if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list].shape[0] != 0:
#                     print(df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list])
                    mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].mean()
                    sum_  = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].sum()
            uid_D15.append([TransactionID_, mean_, sum_])
#             print(TransactionID_, mean_, sum_)
#             if mean_ == 1.0:
#                 print(TransactionID_, mean_, sum_) 
#                 fraud_TransactionIDs.append(TransactionID_)


KeyboardInterrupt: 

In [ ]:
uid_D15 = pd.DataFrame(uid_D15)
uid_D15.columns = ["TransactionID", "mean", "sum"]
uid_D15

### 测试集特征构造

In [ ]:
uid_D15_test = []

In [ ]:
for DAY in tqdm_notebook(range(213, 395+1)):
    for D15 in range(DAY - 182, DAY - 1):  #212
        uid_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), "uid"].values)
        TransactionID_list = list(df.loc[(df["D15"] == D15) & (df["day"] == DAY) & (df["isFraud"] == -1), "TransactionID"].values)
        # print(TransactionID_list)
        for i in range(len(uid_list)):
            TransactionID_ = TransactionID_list[i]
            if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list].shape[0] != 0:
                mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].mean()
                sum_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15), feature_list]["isFraud"].sum()
            else:
                if df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list].shape[0] != 0:
                    mean_ = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].mean()
                    sum_  = df.loc[(df["uid"] == uid_list[i]) & (df["day"] == DAY - D15 - 1), feature_list]["isFraud"].sum()
            uid_D15_test.append([TransactionID_, mean_, sum_])
#             if mean_ == 1.0:
#                 print(TransactionID_, mean_, sum_) 
#                 fraud_TransactionIDs.append(TransactionID_)

                

In [ ]:
uid_D15_test = pd.DataFrame(uid_D15_test)
uid_D15_test.columns = ["TransactionID", "mean", "sum"]
uid_D15_test

In [40]:
df.loc[df["D15"]==0, feature_list]

,uid,isFraud,D15,day,TransactionDT,TransactionID
0,13926_nan_150.0_discover_142.0_credit_315.0_87.0,0,0.0,1,86400,2987000
1,2755_404.0_150.0_mastercard_102.0_credit_325.0...,0,0.0,1,86401,2987001
5,5937_555.0_150.0_visa_226.0_debit_272.0_87.0,0,0.0,1,86510,2987005
6,12308_360.0_150.0_visa_166.0_debit_126.0_87.0,0,0.0,1,86522,2987006
7,12695_490.0_150.0_visa_226.0_debit_325.0_87.0,0,0.0,1,86529,2987007
10,16496_352.0_117.0_mastercard_134.0_credit_nan_nan,0,0.0,1,86549,2987010
11,4461_375.0_185.0_mastercard_224.0_debit_nan_nan,0,0.0,1,86555,2987011
14,11839_490.0_150.0_visa_226.0_debit_226.0_87.0,0,0.0,1,86596,2987014
15,7055_555.0_150.0_visa_226.0_debit_315.0_87.0,0,0.0,1,86618,2987015
18,4663_490.0_150.0_visa_166.0_debit_184.0_87.0,0,0.0,1,86725,2987018
